# Namespaces

[Namespaces](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/) are a way to isolate cluster resources into groups. They're a bit like directories on your computer, but instead of containing files, they contain Kubernetes objects. As you've already learned, every resource in Kubernetes has a name. Some of our names include:
- synergychat-api-configmap
- api-service
- api-deployment
- web-deployment
- ...

You can only use a name once. It is a unique identifier (***Not the same with Python or other language with cause error or replacement***). That's how kubectl apply knows when it should create a new resource and when it should update an existing one. Namespaces allow us to use the same name for different resources, as long as they're in different namespaces.

Run the following command to see namespace list already been use:

```python
kubectl get namespaces
```

or shorter version:

```python
kubectl get ns
```

# Moving Namespaces

Up until this point, we've been working in the `default` namespace. When using `kubectl` commands, you can specify the namespace with the `--namespace` or `-n` flag. If you don't, it will use the `default` namespace.

```
❯ kubectl get pod
NAME                                   READY   STATUS    RESTARTS      AGE
synergychat-api-5966c47bfd-55gtp       1/1     Running   0             4h46m
synergychat-crawler-855497874f-l8xn4   3/3     Running   6 (19h ago)   2d3h
synergychat-web-644b5c88bf-4zd66       1/1     Running   2 (19h ago)   2d6h
synergychat-web-644b5c88bf-7mbsk       1/1     Running   2 (19h ago)   2d6h
synergychat-web-644b5c88bf-nskzs       1/1     Running   2 (19h ago)   2d6h
```

vs.

```
❯ kubectl -n kube-system get pod
NAME                                     READY   STATUS    RESTARTS         AGE
coredns-66bc5c9577-v4jbh                 1/1     Running   15 (19h ago)     13d
coredns-66bc5c9577-vp9wq                 1/1     Running   15 (19h ago)     13d
etcd-docker-desktop                      1/1     Running   15 (19h ago)     13d
kube-apiserver-docker-desktop            1/1     Running   15 (19h ago)     13d
kube-controller-manager-docker-desktop   1/1     Running   15 (19h ago)     13d
kube-proxy-r6dgg                         1/1     Running   15 (19h ago)     13d
kube-scheduler-docker-desktop            1/1     Running   15 (19h ago)     13d
storage-provisioner                      1/1     Running   30 (4h58m ago)   13d
vpnkit-controller                        1/1     Running   15 (19h ago)     13d

The `kube-system` namespace is where all the core Kubernetes components live, it's created automatically when you install Kubernetes. You don't want to mess with it.

### Making a new Namespace

If you have a small cluster with only a few applications, you can probably get away with just using the `default` namespace. However, if you have a large cluster with many applications, and in particular many teams working on those applications, namespaces are a great way to keep things organized.

Create a new namespace called `crawler`:

```python
kubectl create ns crawler

Make sure it was created successfully:

```python
kubectl get ns

Next, add `namespace: crawler` to the `metadata` section of each of the `crawler` resources and apply them. Interestingly, you should see that the resources are "created" not "updated". That's because they're now in a new namespace, and the unique identifier of a resource in Kubernetes is the combination of its name and its namespace.

Make sure your resources are now redeployed in the `crawler` namespace:

```python
kubectl -n crawler get pods
kubectl -n crawler get svc
kubectl -n crawler get configmaps

Then go delete the old resources in the `default` namespace:

```python
kubectl delete deployment <deployment-name>
kubectl delete service <service-name>
kubectl delete configmap <configmap-name>
```

# Intra-Cluster DNS

The front-end of SynergyChat communicates with the `api` application via an external Gateway:

`Domain name http://synchatapi.internal -> Gateway -> service -> pod`

It's now time to connect the `crawler` and `api` applications. The `api` needs to be able to make HTTP requests directly to the `crawler` so that it can get the latest data to power the "stats" slash command.

`front-end -> api -> crawler`

The HTTP communication between the `api` and `crawler` is strictly internal to the cluster, there's no need for an external domain name or Gateway. That makes it simpler, faster and more secure.

### Slash Command

With the tunnel open (`minikube tunnel -c`) open `http://synchat.internal/` in your browser. Post a new message with `/stats` as the message text. You should see a response that says:

```
crawler-bot: Crawler worker not configured
```

That's because the `api` doesn't know how to communicate with the `crawler` yet. Let's fix that.

### DNS

Kubernetes automatically creates [DNS entries](https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/) for each service that can be used to route HTTP traffic between services. The format is:

```
<service-name>.<namespace>.svc.cluster.local
```

Update the `api` application's ConfigMap and Deployment to add a new environment variable called `CRAWLER_BASE_URL` its value should be:

```yaml
http://<service-name>.<namespace>.svc.cluster.local
```

If it's not connecting, you may need to include the port at the end like this:

```yaml
http://<service-name>.<namespace>.svc.cluster.local:8080
```

Replace `<service-name>` and `<namespace>` with the appropriate values for the `crawler` service. This will allow the `api` to make HTTP requests to the `crawler` service.

# Namespace and Routing review

Namespaces are used to separate resources into logical groups. They also impact how internal DNS works.

Each time I use `kubectl` to work with the backend services, I have to specify the namespace:

```python
kubectl -n backend get pods
```

### Internal Routing

Kubernetes makes it really easy for pods to communicate with each other. It does this by automatically creating DNS entries for each service. The format is:

```
<service-name>.<namespace>.svc.cluster.local
```

In reality, the `.svc.cluster.local` isn't needed in most scenarios. If you just use `http://<service-name>.<namespace>` for the api->crawler communication, it will work. When working in the same namespace, you can even just use `http://<service-name>`. That wouldn't work for us in our scenario just because the crawler is in its own separate namespace.

### Internal is Better than External

Unless a service really needs to be made available to the outside world, it's better to keep it internal to the cluster. Internal communications are great because:
- It's faster (assuming nodes are close to each other physically)
- No public DNS is required
- Communication is inherently more secure because it runs on an internal network (usually don't even need HTTPS)

The architecture of SynergyChat is a good example of this. We expose a single JSON API to the outside world, and if the pod that serves those HTTP requests doesn't have all the info it needs locally, it makes internal HTTP requests to other services.